In [1]:
import torch
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.svm import SVR,SVC
from math import sqrt
from sklearn.tree import DecisionTreeClassifier ,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,log_loss,accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

C:\Users\cs040\anaconda3\envs\LAB_work\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\cs040\anaconda3\envs\LAB_work\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\cs040\anaconda3\envs\LAB_work\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [10]:
# 原始資料
pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',50)
pd.set_option('max_colwidth',100)

#給定columns
train_columns = ['ID','TS','X01','X02','X03','X04','X05',
               'X06','X07','X08','X09','X10','X11','X12'
               ,'X13','X14','X15','X16','X17','X18','X19'
                  ,'X20','X21','X22','X23','X24','Y','DI']
test_colums = ['ID','TS','X01','X02','X03','X04','X05',
               'X06','X07','X08','X09','X10','X11','X12'
               ,'X13','X14','X15','X16','X17','X18','X19'
                  ,'X20','X21','X22','X23','X24','DI']


df_train_set = pd.read_csv('fgd_data/train_20545.csv', names=train_columns,skiprows=1,index_col=0)#na_values=' ?'
df_test_set = pd.read_csv('fgd_data/Test_fin.csv', names=test_colums,skiprows=1,index_col=0)#skiprows=1, na_values=' ?'

#刪除不必要欄位
drop_name=['ID','TS']
df_train_set.drop(drop_name, axis=1, inplace=True)
df_test_set.drop(drop_name, axis=1, inplace=True)
df_train_set = df_train_set.dropna()
df_test_set = df_test_set.dropna()


new_cols_train=list(df_train_set.columns)
new_cols_train.remove('Y')

#切分資料集
x_train, y_train = df_train_set[new_cols_train].values, df_train_set['Y'].values
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2,random_state=42)#random_state=42
#x_train, x_validation, y_train, y_validation = df_train_set.iloc[:20144].values,df_train_set.iloc[20144:].values,df_train_set['Y'].iloc[:20144].values,df_train_set['Y'].iloc[20144:].values

x_test = df_test_set.values

df_train_set

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,Y,DI
0,41.83,124.94,263.22,300.57,10.35,10.39,10.32,90.28,261.76,26.72,38.84,955.20,5715.00,5407.50,6407.50,7.37,18.0,3776.31,24.0,0.05,10.35,7.93,78.01,358689.0,3,0
1,41.97,125.30,262.12,300.53,10.33,10.40,10.34,89.50,268.51,26.84,37.54,961.50,5702.50,5407.50,6417.50,7.40,18.0,3760.59,24.0,0.00,9.67,7.97,78.08,339139.0,3,6
2,41.90,125.00,261.45,300.29,10.35,10.41,10.33,89.74,263.78,25.93,34.98,952.19,5690.00,5392.50,6405.00,7.44,18.0,3754.59,24.0,0.00,9.68,7.98,79.50,326082.0,3,6
3,42.33,125.25,262.12,298.94,10.31,10.41,10.33,89.69,260.09,27.34,37.11,937.14,5680.00,5392.50,6427.50,7.47,18.0,3746.73,8.0,5.28,10.92,7.95,80.07,321295.0,3,6
4,42.37,125.03,262.94,301.73,10.33,10.40,10.32,90.20,262.26,41.93,45.88,921.01,5670.00,5377.50,6400.00,7.50,18.0,3748.58,8.0,4.60,11.12,7.94,79.96,339248.0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20540,40.66,124.73,262.06,300.62,10.65,10.53,11.11,154.49,765.99,37.04,45.45,945.13,5735.00,5358.50,5911.38,8.96,32.0,4910.45,9.0,632.11,344.40,7.70,77.49,291896.0,2,6
20541,43.77,126.15,267.76,304.75,10.64,10.57,11.06,157.19,768.79,37.10,45.14,952.87,5717.38,5345.63,5887.13,8.97,32.0,4903.61,9.0,632.89,347.96,7.67,77.98,289563.0,2,6
20542,41.27,124.20,262.99,300.61,10.65,10.57,11.03,157.85,748.09,36.67,45.09,949.86,5722.13,5341.75,5898.88,8.88,32.0,4898.89,9.0,628.78,355.65,7.66,78.58,293656.0,2,6
20543,41.39,124.99,262.99,298.91,10.68,10.59,11.05,156.53,749.15,32.92,42.97,957.37,5725.50,5357.50,5913.63,8.89,32.0,4899.63,9.0,629.43,355.90,7.67,78.62,293181.0,2,6


In [13]:
#模型訓練、調參
xgb = XGBClassifier(n_estimators=90, learning_rate=0.05 ,max_depth=19,eval_metric='mlogloss',subsample=0.9)
xgb_fit = xgb.fit(x_train, y_train)
y_pred = xgb_fit.predict_proba(x_validation)
# xgb_fit = xgb.fit(x_train, y_train)
# y_pred = xgb_fit.predict_proba(x_test)
print(log_loss(y_validation,y_pred))


0.7340625327925455


In [5]:
#xgb
#將預測績效儲存csv
xgb = XGBClassifier(n_estimators=90, learning_rate= 0.05,max_depth=15,eval_metric='mlogloss')
xgb_fit = xgb.fit(x_train, y_train)
y_pred = xgb_fit.predict_proba(x_test)
df_pre = pd.DataFrame(y_pred)
df_submit = pd.read_csv('./fgd_data/submit_samples.csv')
df_submit = df_submit[['ID']]
df_submit = df_submit.join(df_pre) 
df_submit.columns = ['ID', 'C1', 'C2', 'C3', 'C4', 'C5']
df_submit

df_submit.to_csv('C:/Users/cs040/anaconda3/envs/LAB_work/Scripts/fgd_data/xgb_'+str(i)+'.csv',index=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [07:07<00:00, 21.38s/it]


In [ ]:
# RandomForest
#將預測績效儲存csv
clf= RandomForestClassifier(random_state=42,max_depth=18,n_estimators=150,criterion='entropy')#160 180 0.78
R_fit = clf.fit(x_train, y_train)
y_pred = R_fit.predict_proba(x_test)

df_pre = pd.DataFrame(y_pred)
df_submit = pd.read_csv('./fgd_data/submit_samples.csv')
df_submit = df_submit[['ID']]
df_submit = df_submit.join(df_pre) 
df_submit.columns = ['ID', 'C1', 'C2', 'C3', 'C4', 'C5']
df_submit

df_submit.to_csv('C:/Users/cs040/anaconda3/envs/LAB_work/Scripts/fgd_data/xgb_'+str(i)+'.csv',index=0)